In [13]:
import os
import pandas as pd
from mistralai import Mistral
import jsonlines
import json
from dotenv import load_dotenv
load_dotenv() 


True

In [14]:
# 1. Let's get a sample of data

In [15]:
api_key = os.environ["MIXTRAL"]
model = "mistral-large-latest"

In [19]:
df = pd.read_excel("../data/sample_llm.xlsx", engine='openpyxl',index_col=0)

In [20]:
df.shape

(600, 48)

In [21]:
df.head()

,index_new,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,...,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID,Potential_policy
2578,2578,Ji J.; Li Y.; Liu H.; Du Z.; Wei Z.; Shen W.; ...,"Ji, Jiarui (59399138800); Li, Yang (5721870369...",59399138800; 57218703695; 59414895200; 5941478...,SRAP-Agent: Simulating and Optimizing Scarce R...,2024,EMNLP 2024 - 2024 Conference on Empirical Meth...,NaN,NaN,NaN,...,NaN,NaN,English,EMNLP - Conf. Empir. Methods Nat. Lang. Proces...,Conference paper,Final,NaN,Scopus,2-s2.0-85213343894,True
2105,2105,Fakhruzzaman M.N.; Jannah S.Z.; Gunawan S.W.; ...,"Fakhruzzaman, Muhammad Noor (57188878854); Jan...",57188878854; 57222250349; 57222288907; 5720250...,IndoPolicyStats: sentiment analyzer for public...,2024,Bulletin of Electrical Engineering and Informa...,13,1,NaN,...,NaN,NaN,English,Bull. Electr. Eng. Inform.,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85186191889,True
1411,1411,Zeng R.; Zhou D.; Liang Q.; Liu J.; Li H.; Hua...,"Zeng, Runhao (57210900375); Zhou, Dingjie (594...",57210900375; 59482180600; 58482655100; 5948228...,Video2Reward: Generating Reward Function from ...,2024,Frontiers in Artificial Intelligence and Appli...,392,NaN,NaN,...,NaN,NaN,English,Front. Artif. Intell. Appl.,Conference paper,Final,NaN,Scopus,2-s2.0-85216679053,True
3485,3485,Cheng Q.; Yang J.,"Cheng, Quan (57221693268); Yang, Jing (5854993...",57221693268; 58549931200,Allocation and Evolution of Government Attenti...,2023,Sustainability (Switzerland),15,16,12479,...,NaN,NaN,English,Sustainability,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85168801344,True
166,166,Tang S.; Guo Q.,"Tang, Shenghui (59901525700); Guo, Qian (57189...",59901525700; 57189686497,Delayed redevelopment in urban expansion: oppo...,2025,Built Heritage,9,1,22,...,NaN,NaN,English,Built Herit.,Article,Final,NaN,Scopus,2-s2.0-105005401385,True


In [22]:
def make_message_description_instrument(title, abstract):
    title = title.replace("\"", "")
    abstract = abstract.replace("\"", "")
    return [
                {
                    "role": "system",
                    "content": "You are a policy researcher.",
                },
                {
                    "role": "user",
                    "content": f"""
                    You are asked to determined whether an academic publication, given its title and and abstract, is relevant  to the policy studies community. To assess this, please follow the following criteria:

                    1. The title is representative of the content of the abstract.
                    2. Its relevance to the policy studies community is defined as containing information if it advances theoretical, methodological, or empirical understanding of policy processes, instruments, actors, or institutions in the field.


                    You will receive the information in the following format:
                    ### Example input ###
                    TITLE: <title>
                    ABSTRACT: <abstract>
                    ### End example input ###

                    Your answer should be ONLY as "YES" or "NO".

                    ### Example positive answer ###
                    ANSWER: YES
                    ### End example positive answer ###

                    ### Example negative answer ###
                    ANSWER: NO
                    ### End example negative answer ###

                    Given the following abstract from and academic publication:
                     
                    TITLE: {title}
                    ABSTRACT: {abstract}

                    Is this abstract relevant to the policy studies community? 

                    ANSWER:""",
                },
            ]

In [23]:
def get_information_for_mixtral(id, title, abstract):
    message = make_message_description_instrument(title, abstract)
    output_file = f"../data/for_llm/data.jsonl"
    with jsonlines.open(output_file, 'a') as writer:
        writer.write({"custom_id": str(id), "body": {"max_tokens": 50, "messages": message}})
    return

In [24]:
df.apply(lambda x: get_information_for_mixtral(x.index_new, x.Title, x.Abstract), axis=1)

2578    None
2105    None
1411    None
3485    None
166     None
        ... 
6454    None
1428    None
5272    None
7684    None
7769    None
Length: 600, dtype: object

In [6]:
client = Mistral(api_key=api_key)

In [ ]:
batch_data = client.files.upload(
    file={
        "file_name": f"data_for_llm",
        "content": open(f"../data/for_llm/data.jsonl", "rb")},
    purpose = "batch"
    
)

created_job = client.batch.jobs.create(
    input_files=[batch_data.id],
    model="mistral-large-latest",
    endpoint="/v1/chat/completions",
    metadata={"job_type": "response"}
)

In [ ]:
file_id = created_job.id

In [25]:
list_job = client.batch.jobs.list()
list_job.data[0]

BatchJobOut(id='805e0201-e1cd-48f6-92b0-3e8edd927908', input_files=['b76f9baf-409a-441a-9163-55e51a0bd6b4'], endpoint='/v1/chat/completions', model='mistral-large-latest', errors=[], status='SUCCESS', created_at=1751175921, total_requests=1200, completed_requests=1200, succeeded_requests=1200, failed_requests=0, object='batch', metadata={'job_type': 'response'}, output_file='a7e4ad7d-e0fc-4262-9b5a-ef8df1c2103f', error_file=None, started_at=1751175923, completed_at=1751175960)

In [26]:
output_file = list_job.data[0].output_file
output_file

'a7e4ad7d-e0fc-4262-9b5a-ef8df1c2103f'

In [27]:
response = client.files.download(file_id=output_file)

In [28]:
with open("../data/for_llm/results.jsonl", "w") as f:
    for chunk in response.stream:
        f.write(chunk.decode("utf-8"))

In [32]:
def retrieve_json(file_path='../data/for_llm/results.jsonl'):
    tbr = {}
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]
    for i in data:
        id = i["custom_id"]
        choice = i["response"]["body"]["choices"][0]["message"]["content"]
        if "yes" in choice.lower():
            tbr[id] = "1"
        else:
            tbr[id] = "0" 
    return tbr       
    
is_it_relevant = retrieve_json()

In [33]:
df["is_relevant"] = df["index_new"].apply(lambda x: is_it_relevant[str(x)])

In [34]:
df.is_relevant.value_counts()

is_relevant
1    458
0    142
Name: count, dtype: int64

In [35]:
df.to_excel("../data/sample_llm_assessed.xlsx")